In [3]:
from os.path import join, exists
from os import makedirs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

from brainpipe.system import study
from utils import subjects

from fooof.bands import Bands
from fooof.plts.spectra import plot_spectrum, plot_spectrum_shading, plot_spectra_shading
from fooof import FOOOF, FOOOFGroup
from fooof.analysis import get_band_peak_fg
from fooof.objs import fit_fooof_3d
from fooof.objs.utils import average_fg, combine_fooofs, compare_info

from mne.time_frequency import psd_welch
from mne import create_info, EpochsArray
from scipy.stats import mannwhitneyu
import statsmodels.api as sm # import statsmodels 

/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
st = study('Ripples')
st2 = study('Olfacto')
PATH = join(st.path, 'database/R_odors/')
PATH2 = join(st2.path, 'database/Retrieval_No_Odor/')
csv_name = join(PATH, 'fg_models_theta/csv_su=all_roi={}_f=[2-9].csv')
filename = join(PATH, '{}_cond=ALL_odors=ALL_bipo_INFO_clean.npz')
filename2 = join(PATH2, '{}_odor_no_odor_bipo_sel_phys.npz')
PATH_SAVE = join(st.path, 'feature/theta_psd_low_high_no_odor/')
if not exists(PATH_SAVE):
    makedirs(PATH_SAVE)

rois = ['HC','orbital','olf']
sf, fmin, fmax = 500, 1, 14
bands = Bands({'theta' : [4,8]})

for roi in rois:
    df = pd.read_csv(csv_name.format(roi))
    df = df.loc[df['theta_detect']==1]
    nelecs = df.shape[0]
    i = 0
    pows_, CFs_ = np.zeros((nelecs,4)), np.zeros((nelecs,4))
    Ts_od, Ts_2gr, Ts_3gr = np.zeros((nelecs,2)), np.zeros((nelecs,2)),np.zeros((nelecs,2))
    for su,ch in zip(df['subjects'],df['channels']):
        od_mat = np.load(filename.format(su),allow_pickle=True)
        EM_2gr = od_mat['EM_2gr'] #to split trials in Low and High groups
        no_mat = np.load(filename2.format(su),allow_pickle=True)
        
        ch_id = np.where(od_mat['channel']==ch)[0]
        ch_no = np.where(no_mat['channels']==ch)[0]
        if len(ch_no) < 1:
            print(su,ch,'missing elec in no odor')
            i += 1
        else:
            #ntrials, nchannels, npts
            data_od = od_mat['x'][ch_id,...].swapaxes(0,-1).swapaxes(1,2)
            data_no_od = no_mat['x'][ch_no,...].swapaxes(0,-1).swapaxes(1,2)
            
            info = create_info([ch],sfreq=sf,ch_types='eeg')
            epochs = EpochsArray(data_od,info,tmin=-3,verbose=False)
            epochs2 = EpochsArray(data_no_od,info,tmin=-3,verbose=False)
            
            #ntrials, nchannels, nfreqs
            psds, freqs = psd_welch(epochs, fmin=fmin, fmax=fmax, n_jobs=1,
                            tmin=0, tmax=2, n_overlap=150, n_fft=sf*2)
            psds2, _ = psd_welch(epochs2, fmin=fmin, fmax=fmax, n_jobs=1,
                            tmin=0, tmax=2, n_overlap=150, n_fft=sf*2)
            ntrials = psds.shape[0]
            psds2_rand = psds2[np.random.randint(0,psds2.shape[0],size=ntrials),...]

            #Initialize a FOOOF object
            fg = FOOOFGroup(peak_width_limits=[1,8], min_peak_height=0.2, # max_n_peaks=2,
                            peak_threshold=1.,aperiodic_mode='fixed',verbose=False)
            mod_od = fit_fooof_3d(fg,freqs, psds)
            mod_no = fit_fooof_3d(fg,freqs, psds2)
            
            #extract peaks by freq bands
            peak_od = np.squeeze([get_band_peak_fg(fg, bands.theta) for fg in mod_od])
            peak_no = np.squeeze([get_band_peak_fg(fg,bands.theta) for fg in mod_no])
            peak_low = peak_od[np.where(EM_2gr=='low')[0],...]
            peak_high = peak_od[np.where(EM_2gr=='high')[0],...]
            
            #deal with nan values (no theta power detected on this trial)
            peak_no_pow = np.nan_to_num(peak_no[:,1],0)
            peak_od_pow = np.nan_to_num(peak_od[:,1],0)
            peak_l_pow = np.nan_to_num(peak_low[:,1],0)
            peak_h_pow = np.nan_to_num(peak_high[:,1],0)
            
            pows = [peak_no_pow,peak_od_pow,peak_l_pow,peak_h_pow]
            CFs = [peak_no[:,0], peak_od[:,0], peak_low[:,0], peak_high[:,0]]
            #print(np.array([np.mean(x) for x in pows]))
            pows_[i] += np.array([np.median(x) for x in pows])
            CFs_[i] += np.array([np.nanmedian(x) for x in CFs])
            
            Ts_od[i] += mannwhitneyu(peak_no_pow,peak_od[:,1],alternative=None)
            Ts_2gr[i] += mannwhitneyu(peak_low[:,1],peak_high[:,1],alternative=None)
            
            Y = np.concatenate((peak_no_pow,peak_low[:,1],peak_high[:,1]))
            X = [[0]*peak_no_pow.shape[0]+[1]*peak_low.shape[0]+[2]*peak_high.shape[0]]
            X = np.asarray(X).swapaxes(0,1)
            X = sm.add_constant(X)
            results = sm.OLS(Y,X).fit()
            Ts_3gr[i] += np.array([np.round(results.tvalues[1],3),results.pvalues[1]])
            i += 1
    #concat all data and update df
    data = np.concatenate((pows_,CFs_,Ts_od,Ts_2gr,Ts_3gr),axis=1)
    df2 = pd.DataFrame(data,columns=['PW_no','PW_od','PW_l','PW_h',
                                     'CF_no','CF_od','CF_l','CF_h',
                                     'T_od','p_od','T_lh','p_lh',
                                     'T_ols','p_ols'])
    df3 = pd.concat([df,df2],axis=1)
    df3.to_csv(csv_name.format(roi).replace('[2-9].csv','[4-8]_med_allstats.csv'),index=False)          

-> Ripples loaded
-> Olfacto loaded
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


...

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


FERJ j'5-j'4 missing elec in no odor


In [12]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

st = study('Ripples')
PATH = join(st.path, 'database/R_odors/')
csv_name = join(PATH, 'fg_models_theta/csv_su=all_roi={}_f=[4-8]_med_allstats.csv')
path_figs = join(PATH, 'plot_pow_theta_diff/')
figname1 = join(path_figs, 'Mean_pow_all_su_conds={}.png')
figname2 = join(path_figs, 'ratio_sig_pow_all_su_conds={}.png')

if not exists(path_figs):
    makedirs(path_figs)

rois = ['HC','orbital','olf']
conds = ['odor','mem2gr','od_mem']#,'od_mem']

for roi in rois:
    df = pd.read_csv(csv_name.format(roi))
    for cond in conds:
        if cond == 'odor':
            cols = ['subjects','PW_no','PW_od']
        if cond == 'mem2gr':
            cols = ['subjects','PW_l','PW_h']
        if cond == 'od_mem':
            cols = ['subjects','PW_no','PW_l','PW_h']
        df_ = df[cols]
        df_ = df_.dropna()
        nelecs = df_.shape[0]
        subjects = np.concatenate([df_['subjects'].values]*(len(cols)-1))
        PWs = np.concatenate(([df_[col] for col in cols if col!='subjects']))[:,np.newaxis]
        conds_v = np.concatenate([[i]*nelecs for i,col in enumerate(cols) if col!='subjects'])[:,np.newaxis]
        df_res = pd.DataFrame(data=np.concatenate((PWs,conds_v),axis=1),
                              columns=['PWs','conds'],dtype='float')
        
        md = smf.mixedlm("PWs ~ conds", df_res, groups=subjects)
        mdf = md.fit()
        print(df_.mean())
        print(roi, cond, mdf.summary())

-> Ripples loaded
PW_no    0.786468
PW_od    0.792693
dtype: float64
HC odor         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: PWs     
No. Observations: 78      Method:             REML    
No. Groups:       5       Scale:              0.1679  
Min. group size:  2       Likelihood:         -46.6006
Max. group size:  44      Converged:          Yes     
Mean group size:  15.6                                
------------------------------------------------------
              Coef. Std.Err.   z   P>|z| [0.025 0.975]
------------------------------------------------------
Intercept     0.702    0.187 3.747 0.000  0.335  1.069
conds         0.006    0.093 0.067 0.947 -0.176  0.188
Group Var     0.057    0.126                          

PW_l    0.799837
PW_h    0.796622
dtype: float64
HC mem2gr         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: PWs     
No. Observations: 78      Method:             REML    
N

/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [19]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

st = study('Ripples')
PATH = join(st.path, 'database/E_odors/')
csv_name = join(PATH, 'fg_models_theta/csv_su=all_roi={}_f=[2-4]_med_allstats.csv')
path_figs = join(PATH, 'plot_pow_theta_diff/')
figname1 = join(path_figs, 'Mean_pow_all_su_conds={}.png')
figname2 = join(path_figs, 'ratio_sig_pow_all_su_conds={}.png')

if not exists(path_figs):
    makedirs(path_figs)

rois = ['HC','orbital','olf']
conds = ['od_mem']#,'od_mem']

for roi in rois:
    df = pd.read_csv(csv_name.format(roi))
    for cond in conds:
        if cond == 'odor':
            cols = ['subjects','channels','y','PW_no','PW_od','T_od','p_od']
        if cond == 'mem2gr':
            cols = ['subjects','channels','y','PW_l','PW_h','T_lh','p_lh']
        if cond == 'od_mem':
            cols = ['subjects','channels','y','PW_no','PW_l','PW_h','T_ols','p_ols']
        df_ = df[cols]
        df_ = df_.dropna()
        df_['sign'] = np.sign(df_.iloc[:,-3]-df_.iloc[:,-4])
        nelecs = df_.shape[0]
        df_sig = df_.loc[(df_.iloc[:,-2]<0.05)&(df_['sign']>0)]
        nelecs_sig = df_sig.shape[0]
        print(roi,cond,nelecs,nelecs_sig)
        print(df_sig)
        print('\n')

-> Ripples loaded
HC od_mem 40 3
   subjects channels      y     PW_no      PW_l      PW_h  T_ols  \
4      LEFC    d2-d1 -13.65  0.782734  1.058931  1.346973  6.301   
9      FERJ    b4-b3 -13.65  0.803814  0.943560  1.024083  2.909   
42     PIRJ  b'6-b'5 -16.20  0.642992  0.404549  0.529617 -2.339   

           p_ols  sign  
4   1.030106e-09   1.0  
9   3.870614e-03   1.0  
42  1.980750e-02   1.0  


orbital od_mem 48 10
   subjects channels      y     PW_no      PW_l      PW_h   T_ols  \
4      LEFC    o4-o3  37.30  0.499102  1.215216  1.333414   8.989   
5      LEFC    o5-o4  37.85  0.394118  1.138058  1.370249   9.318   
7      LEFC    o7-o6  38.65  0.436789  1.403803  1.515289  11.558   
8      LEFC   o10-o9  39.85  0.699760  1.102448  1.313850   6.453   
11     SEMC    e7-e6  19.75  0.576901  0.795720  1.242931   4.164   
12     SEMC    e8-e7  19.75  0.600611  0.744161  1.178345   2.675   
23     SEMC    u5-u4  43.05  0.511966  0.700663  0.902981   3.690   
30     VACJ  e'4-e'